In [1]:
%pip install -r requirements.txt > /dev/null 2>&1

Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core import SimpleDirectoryReader
import dotenv
import os

dotenv.load_dotenv() 

True

In [3]:
reader = SimpleDirectoryReader(input_dir="obsidian")
documents = reader.load_data()

In [4]:
documents = reader.load_data()
print(f"Loaded {len(documents)} documents.")

Loaded 1 documents.


In [5]:
from llama_index.core import Document
from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_overlap=0),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ]
)

nodes = await pipeline.arun(documents=[Document.example()])

In [6]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./obsidian_chroma_db")
chroma_collection = db.get_or_create_collection("obsidian")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

In [7]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding

embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [11]:
print(index.index_struct)  # This may hold relevant information

IndexDict(index_id='6c0aa67b-5be6-4a12-84e4-590487f9130e', summary=None, nodes_dict={}, doc_id_dict={}, embeddings_dict={})


In [13]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

# Define a function to handle the query
def main():
    llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
    query_engine = index.as_query_engine(
        llm=llm,
        response_mode="tree_summarize",
    )
    result = query_engine.query("What projects am I working on?")
    print("Query Result:", result)  # Print the result
    return result

# Call main and print the results
result = main()
print("Final Output:", result)  # Print final result if needed

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
print(result)